# **1.Importing Libraries**

In [1]:
# Set API keys
import re
import os
import wikipediaapi
from groq import Groq
from langchain.vectorstores import FAISS
from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter

os.environ["GROQ_API_KEY"] = "gsk..."
client = Groq(api_key=os.environ.get("GROQ_API_KEY"))


# **2. Creating an AI Agent**

In [ ]:
class Agent:
    def __init__(self, client, system):
        self.client = client
        self.system = system
        self.memory = []
        # If there is no memory, initialize it with the system message
        if self.memory is not None:
            self.memory = [{"role": "system", "content": self.system}]

    def __call__(self, message=""):
        if message:
            self.memory.append({"role": "user", "content": message})
        result = self.execute()
        self.memory.append({"role": "assistant", "content": result})
        return result

    def execute(self):
        completion = client.chat.completions.create(
            messages = self.memory,
            model="llama-3.3-70b-versatile",
        )   
        return completion.choices[0].message.content

# **Tools/Functions to be used by the Agent**

In [ ]:
wiki = wikipediaapi.Wikipedia(
    language='en',  
    user_agent="aseem"  
)
embeddings = OpenAIEmbeddings()
faiss_store = None
    
def calculate(operation):
    return eval(operation)


def wikipedia_search(query, advanced_query, advanced_search=False, top_k=5):
    global faiss_store
    page = wiki.page(query)

    # Check if the page exists
    if page.exists():
        if advanced_search:
            # Get the full content of the Wikipedia page
            content = page.text
            # Split the content into chunks
            chunks = chunk_text(content)
            # Store the chunks in FAISS
            faiss_store = store_in_faiss(chunks)
            # Retrieve the top-k relevant chunks
            top_k_documents = retrieve_top_k(advanced_query, top_k)
            # Return the retrieved documents
            return f"Context: {" ".join(top_k_documents)}\n"
        else:
            return f"Summary: {page.summary}\n"
    else:
        return f"The page '{query}' does not exist on Wikipedia."


def chunk_text(text, chunk_size=512, chunk_overlap=50):
    """
    Uses LangChain's RecursiveCharacterTextSplitter to chunk the text.
    """
    splitter = RecursiveCharacterTextSplitter(chunk_size=chunk_size, chunk_overlap=chunk_overlap)
    chunks = splitter.split_text(text)
    return chunks


def store_in_faiss(chunks):
    """
    Stores the chunks in a FAISS vector store.
    """
    vector_store = FAISS.from_texts(chunks, embeddings)
    return vector_store


def retrieve_top_k(query, top_k=5):
    """
    Retrieves the top-k most relevant chunks from FAISS.
    """
    if faiss_store is None:
        return "No vector data available. Perform advanced search first."

    # Retrieve top-k documents
    docs_and_scores = faiss_store.similarity_search_with_score(query, top_k)
    top_k_chunks = [doc.page_content for doc, score in docs_and_scores]
    return top_k_chunks

C:\Users\aseem\AppData\Local\Temp\ipykernel_39484\1868225075.py:5: LangChainDeprecationWarning: The class `OpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import OpenAIEmbeddings``.
  embeddings = OpenAIEmbeddings()


# **3. System Prompt**

In [24]:
system_prompt = """
You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

Your available actions are:

1) wikipedia_search:
   e.g. wikipedia_search: query="Albert Einstein", advanced_query="the role of Albert Einstein in quantum mechanics", advanced_search=False, top_k=5
   - If advanced_search is False, you return the top-level summary from the Wikipedia API.
   - If advanced_search is True, you retrieve more detailed context using the provided advanced_query and return the top_k relevant chunks.
   - Make sure that the query looks like a vlaid title for a page on Wikipedia.

2) calculate:
   e.g. calculate: 4.0 * 7 / 3
   - Runs a Python expression and returns the numeric result.

Example session:

Question: Who discovered oxygen and when?
Thought: Let's first try to get a summary from the Wikipedia page.
Action: wikipedia_search: query="Oxygen", advanced_query="Who discovered oxygen and when?", advanced_search=False, top_k=5
PAUSE

You will be called again with this:
Observation: Summary: Oxygen was discovered by Carl Wilhelm Scheele, etc...

Thought: The summary might not include the exact year. Let’s do an advanced search for more detail.
Action: wikipedia_search: query="Oxygen", advanced_query="Who discovered oxygen and when?", advanced_search=True, top_k=5
PAUSE

You will be called again with something like:
Observation: Context: [Detailed paragraphs about Scheele, Priestley, discovery in 1774, etc.]

Thought: Now I have enough information to answer the question.
Answer: Oxygen was discovered by Carl Wilhelm Scheele in 1772, though Joseph Priestley published his findings in 1774.

---

Another example:

Question: What is the sum of 3.14 and 2.86?
Thought: We just need a calculation here.
Action: calculate: 3.14 + 2.86
PAUSE

You will be called again with something like:
Observation: 6.0

Thought: We have the answer.
Answer: 6.0

Now it's your turn:
""".strip()

# **4. Automatic Agentic Flow**

In [10]:
def run_agent(max_iterations=10, query: str = ""):
    agent = Agent(client=client, system=system_prompt)
    tools = ["calculate", "wikipedia_search"]
    next_prompt = query
    i = 0
  
    while i < max_iterations:
        i += 1
        result = agent(next_prompt)
        print(result)

        if "Thought" in result and "Action" in result:
            action = re.findall(r"Action: ([a-z_]+): (.+)", result, re.IGNORECASE)
            chosen_tool = action[0][0]
            args = action[0][1]
            if chosen_tool in tools:
                if chosen_tool == "calculate":
                    tool_result = eval(f"{chosen_tool}({'args'})")
                    next_prompt = f"Observation: {tool_result}"
                else:
                    tool_result = eval(f"{chosen_tool}({args})")
                    next_prompt = f"Observation: {tool_result}"
            else:
                next_prompt = "Observation: Tool not found"

            print(next_prompt)
            continue
        
        if "Answer" in result:
            break

In [11]:
run_agent(query="What is the capital of France and India?")

Thought: To find the capitals of France and India, I can use the Wikipedia API to get information about these countries. I will start by searching for the capital of France.

Action: wikipedia_search: query="France", advanced_query="Capital of France", advanced_search=False, top_k=5
PAUSE
Observation: Summary: France, officially the French Republic, is a country located primarily in Western Europe. Its overseas regions and territories include French Guiana in South America, Saint Pierre and Miquelon in the North Atlantic, the French West Indies, and many islands in Oceania and the Indian Ocean, giving it one of the largest discontiguous exclusive economic zones in the world. Metropolitan France shares borders with Belgium and Luxembourg to the north, Germany to the northeast, Switzerland to the east, Italy and Monaco to the southeast, Andorra and Spain to the south, and a maritime border with the United Kingdom to the northwest. Its metropolitan area extends from the Rhine to the Atlan

# **5. Manual Agentic Flow**


In [59]:
agent = Agent(client=client, system=system_prompt)

In [60]:
result = agent("What is Lagrange Multiplier used for?")
print(result)

Thought: The Lagrange Multiplier is a concept in mathematics, specifically in optimization techniques. To provide a thorough answer, I should first look up the general definition and application of the Lagrange Multiplier on Wikipedia.

Action: wikipedia_search: query="Lagrange multiplier", advanced_query="application of Lagrange multiplier", advanced_search=False, top_k=5
PAUSE


In [61]:
result = wikipedia_search(query="Lagrange Multiplier", advanced_query="use of Lagrange Multipliers", advanced_search=False, top_k=5)
print(result)

Summary: In mathematical optimization, the method of Lagrange multipliers is a strategy for finding the local maxima and minima of a function subject to equation constraints (i.e., subject to the condition that one or more equations have to be satisfied exactly by the chosen values of the variables). It is named after the mathematician Joseph-Louis Lagrange.



In [62]:
next_prompt = "Observation: {}".format(result)
next_prompt

'Observation: Summary: In mathematical optimization, the method of Lagrange multipliers is a strategy for finding the local maxima and minima of a function subject to equation constraints (i.e., subject to the condition that one or more equations have to be satisfied exactly by the chosen values of the variables). It is named after the mathematician Joseph-Louis Lagrange.\n'

In [63]:
result = agent(next_prompt)
print(result)

Thought: The summary provides a good overview of what the Lagrange Multiplier is used for, which is to find the local maxima and minima of a function subject to equation constraints. However, to give a more detailed and specific answer, I would like to know more about the mathematical formulation and examples of its application.

Action: wikipedia_search: query="Lagrange multiplier", advanced_query="Lagrange multiplier method and examples", advanced_search=True, top_k=5
PAUSE


In [64]:
result = wikipedia_search(query="Lagrange Multiplier", advanced_query="use of Lagrange Multipliers", advanced_search=True, top_k=5)
print(result)

Context: In mathematical optimization, the method of Lagrange multipliers is a strategy for finding the local maxima and minima of a function subject to equation constraints (i.e., subject to the condition that one or more equations have to be satisfied exactly by the chosen values of the variables). It is named after the mathematician Joseph-Louis Lagrange. Applying the ordinary Lagrange multiplier method yields The great advantage of this method is that it allows the optimization to be solved without explicit parameterization in terms of the constraints. As a result, the method of Lagrange multipliers is widely used to solve challenging constrained optimization problems. Further, the method of Lagrange multipliers is generalized by the Karush–Kuhn–Tucker conditions, which can also take into account inequality constraints of the form 
  
    
      
        h
        (
        
          x
        
        ) The fact that solutions of the method of Lagrange multipliers are not necessa

In [65]:
next_prompt = "Observation: {}".format(result)
next_prompt

'Observation: Context: In mathematical optimization, the method of Lagrange multipliers is a strategy for finding the local maxima and minima of a function subject to equation constraints (i.e., subject to the condition that one or more equations have to be satisfied exactly by the chosen values of the variables). It is named after the mathematician Joseph-Louis Lagrange. Applying the ordinary Lagrange multiplier method yields The great advantage of this method is that it allows the optimization to be solved without explicit parameterization in terms of the constraints. As a result, the method of Lagrange multipliers is widely used to solve challenging constrained optimization problems. Further, the method of Lagrange multipliers is generalized by the Karush–Kuhn–Tucker conditions, which can also take into account inequality constraints of the form \n  \n    \n      \n        h\n        (\n        \n          x\n        \n        ) The fact that solutions of the method of Lagrange mult

In [66]:
result = agent(next_prompt)
result

'Thought: The observation provides more context and details about the Lagrange Multiplier method, its advantages, and its applications. It seems that the method is widely used to solve constrained optimization problems and has been generalized to include inequality constraints. The observation also mentions some of the challenges and difficulties associated with the method, such as the fact that solutions are not necessarily extrema of the Lagrangian.\n\nWith this information, I can now provide a detailed and specific answer to the question of what the Lagrange Multiplier is used for.\n\nAnswer: The Lagrange Multiplier is a mathematical optimization technique used to find the local maxima and minima of a function subject to equation constraints. It is a strategy for solving constrained optimization problems without explicit parameterization in terms of the constraints. The method is widely used in various fields, including economics, physics, and engineering, to optimize functions subj

In [67]:
for msg in agent.memory:
    print(msg['content'])

You run in a loop of Thought, Action, PAUSE, Observation.
At the end of the loop you output an Answer
Use Thought to describe your thoughts about the question you have been asked.
Use Action to run one of the actions available to you - then return PAUSE.
Observation will be the result of running those actions.

Your available actions are:

Your available actions are:

1) wikipedia_search:
   e.g. wikipedia_search: query="Albert Einstein", advanced_query="the role of Albert Einstein in quantum mechanics", advanced_search=False, top_k=5
   - If advanced_search is False, you return the top-level summary from the Wikipedia API.
   - If advanced_search is True, you retrieve more detailed context using the provided advanced_query and return the top_k relevant chunks.
   - Make sure that the query looks like a vlaid title for a page on Wikipedia.

2) calculate:
   e.g. calculate: 4.0 * 7 / 3
   - Runs a Python expression and returns the numeric result.

Example session:

Question: Who discove